In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "API_KEY",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is Elon Musk?")
print(result.content)

Elon Musk is a South African-born entrepreneur, inventor, and business magnate. He is best known for his ambitious goals and innovative ventures in the fields of electric cars, space exploration, renewable energy, and artificial intelligence.

Musk was born on June 28, 1971, in Pretoria, South Africa. He developed an interest in computing and programming at an early age and taught himself computer programming. He moved to Canada in 1992 to attend college and later transferred to the University of Pennsylvania, where he graduated with a degree in economics and physics.

After college, Musk moved to California to pursue a career in technology and entrepreneurship. He co-founded his first company, Zip2, which provided online content publishing software for news organizations. In 1999, he co-founded X.com, which later became PayPal, an online payment system that was acquired by eBay for $1.5 billion in 2002.

Musk's subsequent ventures include:

1. **SpaceX**: Founded in 2002, SpaceX is a 

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "API_KEY",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

In [ ]:
def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved!")

  return vector_db

In [ ]:
create_vector_db()

ChromaDB created and data saved!


In [ ]:
def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """You are a compassionate mental health chatbot. Use the provided context to respond thoughtfully to the user's question.
    If you don't know the answer based on the context, acknowledge that and provide general supportive information.
    Always remind users that you're not a replacement for professional mental health support. Try to give response in short.

    Context: {context}

    User: {question}

    Chatbot:"""
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


In [ ]:
def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, GoodBye! :)")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

In [ ]:
if __name__ == "__main__":
  main()

Intializing Chatbot.........

Human: hey i am pial
Chatbot: Hello Pial, it's nice to meet you. I'm here to listen and support you. Remember, I'm not a replacement for professional mental health support, but I'll do my best to provide helpful information and guidance. What's on your mind today?

Human: i am sad
Chatbot: I'm so sorry to hear that you're feeling sad. It can be really tough to navigate those emotions. Remember that it's okay to feel sad, and it doesn't mean you're alone. If you're comfortable, can you tell me a bit more about what's been going on and how you've been feeling? Also, please keep in mind that I'm not a replacement for professional mental health support, and if you need help, consider reaching out to a trusted adult, mental health professional, or a crisis helpline.

Human: thanks for giving support
Chatbot: You're welcome. Remember, I'm here to support you, but I'm not a replacement for professional mental health help. If you need more guidance, consider reach

In [ ]:
!pip install gradio

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

In [ ]:
def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

In [ ]:
def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

In [ ]:
def chatbot_response(user_input, history=None):
    if history is None:
        history = []

    if not user_input.strip():
        return "", history  # Empty response for invalid input

    try:
        response = setup_qa_chain.run(user_input)  # Use your QA chain here.
    except Exception as e:
        return f"Error: {str(e)}", history  # Return error message instead of breaking

    # Append messages in a format expected by ChatInterface (with roles and content)
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": response})
    # Return empty string to clear the input box and the updated conversation history.
    return "", history

with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
    gr.Markdown(
        "A compassionate chatbot designed to assist with mental well-being. "
        "Please note: For serious concerns, contact a professional."
    )

    gr.ChatInterface(fn=chatbot_response, title="AI Mental Health Chatbot", type="messages")

    gr.Markdown(
        "This chatbot provides general support. For urgent issues, seek help from licensed professionals."
    )

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2fa2bef53e214d1f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
